<a href="https://colab.research.google.com/github/Aishwarya240/AIES_LAB/blob/main/Exp_01_AIES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
yasserh_titanic_dataset_path = kagglehub.dataset_download('yasserh/titanic-dataset')

print('Data source import complete.')


Data source import complete.


In [ ]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# -- model --
from sklearn.linear_model import LogisticRegression

In [ ]:
!pip install fairlearn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 6.4 MB/s eta 0:00:00


In [ ]:
from fairlearn.metrics import MetricFrame, selection_rate

In [ ]:
url = '/kaggle/input/titanic-dataset/Titanic-Dataset.csv'
data = pd.read_csv(url)

In [ ]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
data.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [ ]:
features = ["Survived", "Pclass", "Sex", "Age", "SibSp", "Parch", "Fare"]
data = data[features]

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
dtypes: float64(2), int64(4), object(1)
memory usage: 48.9+ KB


In [ ]:
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,male,22.0,1,0,7.2500
1,1,1,female,38.0,1,0,71.2833
2,1,3,female,26.0,0,0,7.9250
3,1,1,female,35.0,1,0,53.1000
4,0,3,male,35.0,0,0,8.0500


In [ ]:
# If 'data' is a slice, ensure it's a copy first
data = data.copy()

# Convert columns to string directly
data["Survived"] = data["Survived"].astype(str)
data["Sex"] = data["Sex"].astype(str)


In [ ]:
data

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,male,22.0,1,0,7.2500
1,1,1,female,38.0,1,0,71.2833
2,1,3,female,26.0,0,0,7.9250
3,1,1,female,35.0,1,0,53.1000
4,0,3,male,35.0,0,0,8.0500
...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000
887,1,1,female,19.0,0,0,30.0000
888,0,3,female,NaN,1,2,23.4500
889,1,1,male,26.0,0,0,30.0000


In [ ]:
data.isnull().sum()

,0
Survived,0
Pclass,0
Sex,0
Age,177
SibSp,0
Parch,0
Fare,0


In [ ]:
data = data.dropna()

In [ ]:
df = pd.get_dummies(data, drop_first=True)
df.head()

,Pclass,Age,SibSp,Parch,Fare,Survived_1,Sex_male
0,3,22.0,1,0,7.2500,False,True
1,1,38.0,1,0,71.2833,True,False
2,3,26.0,0,0,7.9250,True,False
3,1,35.0,1,0,53.1000,True,False
4,3,35.0,0,0,8.0500,False,True


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 714 entries, 0 to 890
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Pclass      714 non-null    int64  
 1   Age         714 non-null    float64
 2   SibSp       714 non-null    int64  
 3   Parch       714 non-null    int64  
 4   Fare        714 non-null    float64
 5   Survived_1  714 non-null    bool   
 6   Sex_male    714 non-null    bool   
dtypes: bool(2), float64(2), int64(3)
memory usage: 34.9 KB


In [ ]:
X = df.drop(["Survived_1"], axis=1)
y = df["Survived_1"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [ ]:
y_pred = model.predict(X_test)
y_pred

array([False, False,  True,  True, False, False, False, False,  True,
        True,  True,  True, False, False,  True, False, False,  True,
       False,  True, False,  True,  True, False, False, False, False,
       False,  True, False,  True,  True,  True, False, False, False,
       False, False, False, False, False, False,  True, False, False,
        True,  True, False, False,  True, False,  True,  True, False,
       False,  True,  True, False, False, False,  True, False,  True,
       False, False, False, False,  True, False,  True, False,  True,
       False, False, False, False, False, False,  True,  True, False,
       False, False, False, False,  True, False, False, False, False,
       False,  True,  True, False,  True,  True,  True,  True, False,
       False,  True,  True, False,  True,  True, False, False, False,
       False,  True, False, False, False, False, False,  True, False,
        True, False, False,  True, False, False, False,  True, False,
       False,  True,

In [ ]:
# Fairness evaluation
sex = X_test['Sex_male']
metric_frame = MetricFrame(metrics=selection_rate,
y_true=y_test,
y_pred=y_pred,
sensitive_features=sex)
print("Selection Rates by Sex Male:\n", metric_frame.by_group)

Selection Rates by Sex Male:
 Sex_male
False    0.841463
True     0.067669
Name: selection_rate, dtype: float64


- Based on the selection rates, it appears that there is a significant disparity in the selection rate for "Sex_male=False" (which represents females) compared to "Sex_male=True" (which represents males).
- The selection rate for females is 0.841, while for males it is 0.067. This indicates that the model is much more likely to predict 'Survived' (selection rate close to 1) for females than for males.